In [1]:
!nvidia-smi

Tue Nov 30 01:42:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 455.38       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3080    Off  | 00000000:01:00.0  On |                  N/A |
| 78%   53C    P0    39W / 340W |    252MiB / 10018MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
import torch_geometric
from torch_geometric.datasets import ModelNet
from torch_geometric.transforms import SamplePoints
from torch_geometric.transforms import Compose
from torch_geometric.transforms import LinearTransformation
from torch_geometric.transforms import GenerateMeshNormals
from torch_geometric.transforms import NormalizeScale
from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch
from torch_scatter import scatter_mean
import sys


## NOTE:
# Made it work with ModelNet10 (in folder "data/ModelNet") that has only 10 classes
# Now I'll try to test it with ModelNet40 ("data/ModelNet40") with 40 classes

# ----------------------------------------------------------------
# Choosing device:
device = "cuda" if torch.cuda.is_available() else "cpu"

# ----------------------------------------------------------------
# Hyper parameters:
num_points = 1024    # 1024 seems to be the limit...?
batch_size = 32      # not yet used
num_epochs = 20
learning_rate = 0.001

##########################################################################
#                  NO LONGER USED but leave them here...
F = [128, 512, 1024]  # Number of graph convolutional filters.
K = [6, 5, 3]  # Polynomial orders.
M = [512, 128, 10]  # Output dimensionality of fully connected layers.
##########################################################################

# ------------------------------------------------------------------------
## Data loading:
# For ModelNet10 change root to "data/ModelNet"   -> 10 classes
# For MpdelNet40 change root to "data/ModelNet40" -> 40 classes
# Don't forget to change accordingly the output layer from the model...
transforms = Compose([SamplePoints(num_points, include_normals=True), NormalizeScale()])
dataset_train = ModelNet(root="data/ModelNet40", name='40', train=True, transform=transforms)
dataset_test = ModelNet(root="data/ModelNet40", name='40', train=False, transform=transforms)

# Shuffle Data
dataset_train = dataset_train.shuffle()
dataset_test = dataset_test.shuffle()

# Verification...
print(f"Train dataset shape: {dataset_train}")
print(f"Test dataset shape:  {dataset_test}")

print(dataset_train[0])

Train dataset shape: ModelNet40(9843)
Test dataset shape:  ModelNet40(2468)
Data(pos=[1024, 3], y=[1], normal=[1024, 3])


In [3]:
import torch.nn as nn
import torch_geometric.utils as utils
import torch_geometric.nn.conv as conv

class GetGraph(nn.Module):
    def __init__(self):
        """
        Creates the weighted adjacency matrix 'W'
        Taked directly from RGCNN
        """
        super(GetGraph, self).__init__()

    def forward(self, point_cloud):
        point_cloud_transpose = point_cloud.permute(0, 2, 1)
        point_cloud_inner = torch.matmul(point_cloud, point_cloud_transpose)
        point_cloud_inner = -2 * point_cloud_inner
        point_cloud_square = torch.sum(torch.mul(point_cloud, point_cloud), dim=2, keepdim=True)
        point_cloud_square_tranpose = point_cloud_square.permute(0, 2, 1)
        adj_matrix = point_cloud_square + point_cloud_inner + point_cloud_square_tranpose
        adj_matrix = torch.exp(-adj_matrix)
        return adj_matrix


class GetLaplacian(nn.Module):
    def __init__(self, normalize=True):
        """
        Computes the Graph Laplacian from a Weighted Graph
        Taken directly from RGCNN - currently not used - might need to find alternatives in PyG for loss function
        """
        super(GetLaplacian, self).__init__()
        self.normalize = normalize

        def diag(self, mat):
        # input is batch x vertices
            d = []
            for vec in mat:
                d.append(torch.diag(vec))
            return torch.stack(d)

    def forward(self, adj_matrix):
        if self.normalize:
            D = torch.sum(adj_matrix, dim=1)
            eye = torch.ones_like(D)
            eye = self.diag(eye)
            D = 1 / torch.sqrt(D)
            D = self.diag(D)
            L = eye - torch.matmul(torch.matmul(D, adj_matrix), D)
        else:
            D = torch.sum(adj_matrix, dim=1)
            D = torch.diag(D)
            L = D - adj_matrix
        return L

In [4]:
import torch.nn as nn
import torch_geometric.utils as utils
import torch_geometric.nn.conv as conv

class RGCNN_model(nn.Module):
    def __init__(self, vertice, F, K, M, regularization = 0, dropout = 0):
        # verify the consistency w.r.t. the number of layers
        assert len(F) == len(K)
        super(RGCNN_model, self).__init__()
        '''
        F := List of Convolutional Layers dimensions
        K := List of Chebyshev polynomial degrees
        M := List of Fully Connected Layers dimenstions
        
        Currently the dimensions are 'hardcoded'
        '''
        self.F = F
        self.K = K
        self.M = M

        self.vertice = vertice
        self.regularization = regularization
        self.dropout = dropout

        # initialize the model layers
        self.get_graph = GetGraph()
        self.get_laplacian = GetLaplacian(normalize=True)
        self.pool = nn.MaxPool1d(vertice)
        self.relu = nn.ReLU()
        
        ###################################################################
        #                                 CHANGE HERE 
        self.conv1 = conv.ChebConv(6, 128, 6)
        self.conv2 = conv.ChebConv(128, 512, 5)
        self.conv3 = conv.ChebConv(512, 1024, 3)

        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 40)
        ###################################################################
        '''
        for i in range(len(F)):
            if i == 0:
                layer = ChebConv(Fin=3, K=K[i], Fout=F[i])
            else:
                layer = ChebConv(Fin=F[i-1], K=K[i], Fout=F[i])
            setattr(self, 'gcn%d'%i, layer)
        for i in range(len(M)):
            if i==0:
                layer = nn.Linear(F[-1], M[i])
            else:
                layer = nn.Linear(M[i-1], M[i])
            setattr(self, 'fc%d'%i, layer)
        '''

    def forward(self, x):
        # forward pass
        L   = self.get_graph(x.detach())  # we don't want to compute gradients when building the graph
        edge_index, edge_weight = utils.dense_to_sparse(L)
        out = self.conv1(x, edge_index, edge_weight)
        out = self.relu(out)

        L   = self.get_graph(out.detach())
        edge_index, edge_weight = utils.dense_to_sparse(L)

        out = self.conv2(out, edge_index, edge_weight)
        out = self.relu(out)

        L   = self.get_graph(out.detach())
        edge_index, edge_weight = utils.dense_to_sparse(L)

        out = self.conv3(out, edge_index, edge_weight)
        out = self.relu(out)

        '''
        for i in range(len(self.F)):
            x = getattr(self, 'gcn%d'%i)(x, L)
            print(x)
            x = self.relu(x)
        '''

        out = out.permute(0, 2, 1) # Transpose
        out = self.pool(out)
        out.squeeze_(2)

        out = self.fc1(out)
        out = self.relu(out)

        out = self.fc2(out)
        out = self.relu(out)

        out = self.fc3(out)
        return out
        '''
        for i in range(len(self.M)):
            x = getattr(self, "fc%d"%i)(x)
        return x
        '''

model = RGCNN_model(num_points, F, K, M)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
conv1.bias 	 torch.Size([128])
conv1.lins.0.weight 	 torch.Size([128, 6])
conv1.lins.1.weight 	 torch.Size([128, 6])
conv1.lins.2.weight 	 torch.Size([128, 6])
conv1.lins.3.weight 	 torch.Size([128, 6])
conv1.lins.4.weight 	 torch.Size([128, 6])
conv1.lins.5.weight 	 torch.Size([128, 6])
conv2.bias 	 torch.Size([512])
conv2.lins.0.weight 	 torch.Size([512, 128])
conv2.lins.1.weight 	 torch.Size([512, 128])
conv2.lins.2.weight 	 torch.Size([512, 128])
conv2.lins.3.weight 	 torch.Size([512, 128])
conv2.lins.4.weight 	 torch.Size([512, 128])
conv3.bias 	 torch.Size([1024])
conv3.lins.0.weight 	 torch.Size([1024, 512])
conv3.lins.1.weight 	 torch.Size([1024, 512])
conv3.lins.2.weight 	 torch.Size([1024, 512])
fc1.weight 	 torch.Size([512, 1024])
fc1.bias 	 torch.Size([512])
fc2.weight 	 torch.Size([128, 512])
fc2.bias 	 torch.Size([128])
fc3.weight 	 torch.Size([40, 128])
fc3.bias 	 torch.Size([40])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'betas

In [5]:
# Training
loss = torch.nn.CrossEntropyLoss()
correct_percentage_list = []
model.train()
for epoch in range(num_epochs):
    correct = 0
    for i, data in enumerate(dataset_train):
        # make sure the gradients are empty
        optimizer.zero_grad()
        
        # Data preparation 
        pos = data.pos        # (num_points * 3)   
        normals = data.normal # (num_points * 3)
        x = torch.cat([pos, normals], dim=1)   # (num_points * 6)
        x = x.unsqueeze(0)    # (1 * num_points * 6)     the first dimension may be used for batching?
        x = x.type(torch.float32)  # other types of data may be unstable

        y = data.y              # (1)
        y = y.type(torch.long)  # required by the loss function
        
        x = x.to(device)      # to CUDA if available
        y = y.to(device)
     
        # Forward pass
        y_pred = model(x)     # (1 * 40)

        class_pred = torch.argmax(y_pred.squeeze(0))  # (1)  
        correct += int((class_pred == y).sum())       # to compute the accuracy for each epoch
        
        # loss and backward
        l = loss(y_pred, y)   # one value
        l.backward()          # update gradients
        
        # optimisation
        optimizer.step()
    
        if i%100==0:
            print(f"Epoch: {epoch}, Sample: {i}, Loss:{l} - Predicted class vs Real Cass: {class_pred} <-> {y.item()}")
    print(f"~~~~~~~~~ CORRECT: {correct / len(dataset_train)} ~~~~~~~~~~~")
    correct_percentage_list.append(correct / len(dataset_train))
print(correct_percentage_list)

torch.save(model.state_dict(), "/home/alex/Alex_pyt_geom/models")

with torch.no_grad():
    model.eval()
    correct = 0
    for data in dataset_test:
        pos = data.pos        # (num_points * 3)   
        normals = data.normal # (num_points * 3)
        x = torch.cat([pos, normals], dim=1)   # (num_points * 6)
        x = x.unsqueeze(0)    # (1 * num_points * 6)     the first dimension may be used for batching?
        x = x.type(torch.float32)  # other types of data may be unstable

        y = data.y              # (1)
        y = y.type(torch.long)  # required by the loss function
        
        x = x.to(device)      # to CUDA if available
        y = y.to(device)
     
        # Forward pass
        y_pred = model(x)     # (1 * 40)

        class_pred = torch.argmax(y_pred)
        correct += int((class_pred == y).sum())

    print(f"Correct percentage : {correct / len(dataset_test)}")

Epoch: 0, Sample: 0, Loss:3.605333089828491 - Predicted class vs Real Cass: 2 <-> 17
Epoch: 0, Sample: 100, Loss:5.3537774085998535 - Predicted class vs Real Cass: 8 <-> 20
Epoch: 0, Sample: 200, Loss:2.233015537261963 - Predicted class vs Real Cass: 8 <-> 8
Epoch: 0, Sample: 300, Loss:3.7359702587127686 - Predicted class vs Real Cass: 2 <-> 36
Epoch: 0, Sample: 400, Loss:2.681027412414551 - Predicted class vs Real Cass: 2 <-> 2
Epoch: 0, Sample: 500, Loss:3.349234104156494 - Predicted class vs Real Cass: 8 <-> 25
Epoch: 0, Sample: 600, Loss:4.106689929962158 - Predicted class vs Real Cass: 30 <-> 31
Epoch: 0, Sample: 700, Loss:3.887910842895508 - Predicted class vs Real Cass: 8 <-> 17
Epoch: 0, Sample: 800, Loss:3.859363079071045 - Predicted class vs Real Cass: 30 <-> 34
Epoch: 0, Sample: 900, Loss:4.74408483505249 - Predicted class vs Real Cass: 0 <-> 26
Epoch: 0, Sample: 1000, Loss:3.600234031677246 - Predicted class vs Real Cass: 30 <-> 9
Epoch: 0, Sample: 1100, Loss:1.306845426559

Epoch: 0, Sample: 9300, Loss:0.1504897177219391 - Predicted class vs Real Cass: 22 <-> 22
Epoch: 0, Sample: 9400, Loss:1.5665549039840698 - Predicted class vs Real Cass: 2 <-> 30
Epoch: 0, Sample: 9500, Loss:1.8872056007385254 - Predicted class vs Real Cass: 33 <-> 12
Epoch: 0, Sample: 9600, Loss:0.28281089663505554 - Predicted class vs Real Cass: 8 <-> 8
Epoch: 0, Sample: 9700, Loss:0.11119053512811661 - Predicted class vs Real Cass: 22 <-> 22
Epoch: 0, Sample: 9800, Loss:4.547256946563721 - Predicted class vs Real Cass: 4 <-> 31
~~~~~~~~~ CORRECT: 0.39916692065427206 ~~~~~~~~~~~
Epoch: 1, Sample: 0, Loss:0.2818385064601898 - Predicted class vs Real Cass: 17 <-> 17
Epoch: 1, Sample: 100, Loss:1.6615036725997925 - Predicted class vs Real Cass: 34 <-> 20
Epoch: 1, Sample: 200, Loss:0.00022623363474849612 - Predicted class vs Real Cass: 8 <-> 8
Epoch: 1, Sample: 300, Loss:2.214076042175293 - Predicted class vs Real Cass: 21 <-> 36
Epoch: 1, Sample: 400, Loss:0.2576720714569092 - Predicte

Epoch: 1, Sample: 8600, Loss:0.0 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 1, Sample: 8700, Loss:0.17097751796245575 - Predicted class vs Real Cass: 30 <-> 30
Epoch: 1, Sample: 8800, Loss:1.0127103328704834 - Predicted class vs Real Cass: 15 <-> 26
Epoch: 1, Sample: 8900, Loss:2.71553897857666 - Predicted class vs Real Cass: 16 <-> 36
Epoch: 1, Sample: 9000, Loss:0.02781367115676403 - Predicted class vs Real Cass: 22 <-> 22
Epoch: 1, Sample: 9100, Loss:1.3351351299206726e-05 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 1, Sample: 9200, Loss:3.2294178009033203 - Predicted class vs Real Cass: 26 <-> 24
Epoch: 1, Sample: 9300, Loss:0.003279072232544422 - Predicted class vs Real Cass: 22 <-> 22
Epoch: 1, Sample: 9400, Loss:0.09524322301149368 - Predicted class vs Real Cass: 30 <-> 30
Epoch: 1, Sample: 9500, Loss:1.105514407157898 - Predicted class vs Real Cass: 33 <-> 12
Epoch: 1, Sample: 9600, Loss:0.00014840454969089478 - Predicted class vs Real Cass: 8 <-> 8
Epoch: 1, Sample: 970

Epoch: 2, Sample: 7900, Loss:1.9072210788726807 - Predicted class vs Real Cass: 14 <-> 23
Epoch: 2, Sample: 8000, Loss:0.015823161229491234 - Predicted class vs Real Cass: 18 <-> 18
Epoch: 2, Sample: 8100, Loss:1.7332231998443604 - Predicted class vs Real Cass: 14 <-> 39
Epoch: 2, Sample: 8200, Loss:0.0 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 2, Sample: 8300, Loss:1.322779655456543 - Predicted class vs Real Cass: 21 <-> 25
Epoch: 2, Sample: 8400, Loss:0.0012490098597481847 - Predicted class vs Real Cass: 22 <-> 22
Epoch: 2, Sample: 8500, Loss:3.3275861740112305 - Predicted class vs Real Cass: 17 <-> 29
Epoch: 2, Sample: 8600, Loss:0.0 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 2, Sample: 8700, Loss:0.0002019201492657885 - Predicted class vs Real Cass: 30 <-> 30
Epoch: 2, Sample: 8800, Loss:0.7242251038551331 - Predicted class vs Real Cass: 26 <-> 26
Epoch: 2, Sample: 8900, Loss:2.1135354042053223 - Predicted class vs Real Cass: 21 <-> 36
Epoch: 2, Sample: 9000, Loss:0.002133

Epoch: 3, Sample: 7200, Loss:0.0006211258587427437 - Predicted class vs Real Cass: 8 <-> 8
Epoch: 3, Sample: 7300, Loss:0.3588687777519226 - Predicted class vs Real Cass: 25 <-> 25
Epoch: 3, Sample: 7400, Loss:0.22318971157073975 - Predicted class vs Real Cass: 9 <-> 9
Epoch: 3, Sample: 7500, Loss:3.641617774963379 - Predicted class vs Real Cass: 3 <-> 34
Epoch: 3, Sample: 7600, Loss:0.0017897554207593203 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 3, Sample: 7700, Loss:3.4258155822753906 - Predicted class vs Real Cass: 14 <-> 38
Epoch: 3, Sample: 7800, Loss:0.022182907909154892 - Predicted class vs Real Cass: 4 <-> 4
Epoch: 3, Sample: 7900, Loss:2.3921992778778076 - Predicted class vs Real Cass: 14 <-> 23
Epoch: 3, Sample: 8000, Loss:0.01212382223457098 - Predicted class vs Real Cass: 18 <-> 18
Epoch: 3, Sample: 8100, Loss:4.548727989196777 - Predicted class vs Real Cass: 14 <-> 39
Epoch: 3, Sample: 8200, Loss:4.768370445162873e-07 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 3, 

...

Epoch: 15, Sample: 9000, Loss:2.4376230239868164 - Predicted class vs Real Cass: 39 <-> 22
Epoch: 15, Sample: 9100, Loss:2.50339189733495e-06 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 15, Sample: 9200, Loss:18.660411834716797 - Predicted class vs Real Cass: 26 <-> 24
Epoch: 15, Sample: 9300, Loss:0.0 - Predicted class vs Real Cass: 22 <-> 22
Epoch: 15, Sample: 9400, Loss:0.0 - Predicted class vs Real Cass: 30 <-> 30
Epoch: 15, Sample: 9500, Loss:3.0615458488464355 - Predicted class vs Real Cass: 27 <-> 12
Epoch: 15, Sample: 9600, Loss:1.4305104514278355e-06 - Predicted class vs Real Cass: 8 <-> 8
Epoch: 15, Sample: 9700, Loss:0.0 - Predicted class vs Real Cass: 22 <-> 22
Epoch: 15, Sample: 9800, Loss:6.877470016479492 - Predicted class vs Real Cass: 4 <-> 31
~~~~~~~~~ CORRECT: 0.7237630803616784 ~~~~~~~~~~~
Epoch: 16, Sample: 0, Loss:0.0 - Predicted class vs Real Cass: 17 <-> 17
Epoch: 16, Sample: 100, Loss:0.0 - Predicted class vs Real Cass: 20 <-> 20
Epoch: 16, Sample: 200, Loss

Epoch: 16, Sample: 8500, Loss:6.790902614593506 - Predicted class vs Real Cass: 17 <-> 29
Epoch: 16, Sample: 8600, Loss:0.0 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 16, Sample: 8700, Loss:5.3881147323409095e-05 - Predicted class vs Real Cass: 30 <-> 30
Epoch: 16, Sample: 8800, Loss:0.2518519461154938 - Predicted class vs Real Cass: 26 <-> 26
Epoch: 16, Sample: 8900, Loss:3.073362350463867 - Predicted class vs Real Cass: 31 <-> 36
Epoch: 16, Sample: 9000, Loss:0.0 - Predicted class vs Real Cass: 22 <-> 22
Epoch: 16, Sample: 9100, Loss:0.0 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 16, Sample: 9200, Loss:3.964416027069092 - Predicted class vs Real Cass: 32 <-> 24
Epoch: 16, Sample: 9300, Loss:0.0 - Predicted class vs Real Cass: 22 <-> 22
Epoch: 16, Sample: 9400, Loss:0.01934775710105896 - Predicted class vs Real Cass: 30 <-> 30
Epoch: 16, Sample: 9500, Loss:2.630918264389038 - Predicted class vs Real Cass: 2 <-> 12
Epoch: 16, Sample: 9600, Loss:0.0 - Predicted class vs Real Cas

Epoch: 17, Sample: 8100, Loss:1.2174345254898071 - Predicted class vs Real Cass: 39 <-> 39
Epoch: 17, Sample: 8200, Loss:0.0 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 17, Sample: 8300, Loss:1.2492071390151978 - Predicted class vs Real Cass: 25 <-> 25
Epoch: 17, Sample: 8400, Loss:0.0 - Predicted class vs Real Cass: 22 <-> 22
Epoch: 17, Sample: 8500, Loss:18.411788940429688 - Predicted class vs Real Cass: 18 <-> 29
Epoch: 17, Sample: 8600, Loss:0.0 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 17, Sample: 8700, Loss:2.9802276912960224e-06 - Predicted class vs Real Cass: 30 <-> 30
Epoch: 17, Sample: 8800, Loss:0.47406482696533203 - Predicted class vs Real Cass: 26 <-> 26
Epoch: 17, Sample: 8900, Loss:3.388969898223877 - Predicted class vs Real Cass: 34 <-> 36
Epoch: 17, Sample: 9000, Loss:2.661038875579834 - Predicted class vs Real Cass: 39 <-> 22
Epoch: 17, Sample: 9100, Loss:0.0 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 17, Sample: 9200, Loss:4.3912153244018555 - Predicted c

Epoch: 18, Sample: 7600, Loss:0.0 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 18, Sample: 7700, Loss:2.3018908500671387 - Predicted class vs Real Cass: 25 <-> 38
Epoch: 18, Sample: 7800, Loss:0.01605921797454357 - Predicted class vs Real Cass: 4 <-> 4
Epoch: 18, Sample: 7900, Loss:2.1938552856445312 - Predicted class vs Real Cass: 14 <-> 23
Epoch: 18, Sample: 8000, Loss:0.005059652961790562 - Predicted class vs Real Cass: 18 <-> 18
Epoch: 18, Sample: 8100, Loss:1.476630687713623 - Predicted class vs Real Cass: 14 <-> 39
Epoch: 18, Sample: 8200, Loss:0.0 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 18, Sample: 8300, Loss:2.8725714683532715 - Predicted class vs Real Cass: 4 <-> 25
Epoch: 18, Sample: 8400, Loss:3.933898824470816e-06 - Predicted class vs Real Cass: 22 <-> 22
Epoch: 18, Sample: 8500, Loss:2.7486319541931152 - Predicted class vs Real Cass: 18 <-> 29
Epoch: 18, Sample: 8600, Loss:0.0 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 18, Sample: 8700, Loss:0.0 - Predicted cl

Epoch: 19, Sample: 7100, Loss:0.0 - Predicted class vs Real Cass: 2 <-> 2
Epoch: 19, Sample: 7200, Loss:0.001122321467846632 - Predicted class vs Real Cass: 8 <-> 8
Epoch: 19, Sample: 7300, Loss:1.1920928244535389e-07 - Predicted class vs Real Cass: 25 <-> 25
Epoch: 19, Sample: 7400, Loss:0.3000130355358124 - Predicted class vs Real Cass: 9 <-> 9
Epoch: 19, Sample: 7500, Loss:2.3972079753875732 - Predicted class vs Real Cass: 29 <-> 34
Epoch: 19, Sample: 7600, Loss:0.0 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 19, Sample: 7700, Loss:3.4657063484191895 - Predicted class vs Real Cass: 36 <-> 38
Epoch: 19, Sample: 7800, Loss:0.004538354929536581 - Predicted class vs Real Cass: 4 <-> 4
Epoch: 19, Sample: 7900, Loss:1.3062236309051514 - Predicted class vs Real Cass: 14 <-> 23
Epoch: 19, Sample: 8000, Loss:0.0 - Predicted class vs Real Cass: 18 <-> 18
Epoch: 19, Sample: 8100, Loss:1.6041102409362793 - Predicted class vs Real Cass: 14 <-> 39
Epoch: 19, Sample: 8200, Loss:0.0 - Predicted 

In [6]:
with torch.no_grad():
    model.eval()
    correct = 0
    for data in dataset_test:
        pos = data.pos        # (num_points * 3)   
        normals = data.normal # (num_points * 3)
        x = torch.cat([pos, normals], dim=1)   # (num_points * 6)
        x = x.unsqueeze(0)    # (1 * num_points * 6)     the first dimension may be used for batching?
        x = x.type(torch.float32)  # other types of data may be unstable

        y = data.y              # (1)
        y = y.type(torch.long)  # required by the loss function
        
        x = x.to(device)      # to CUDA if available
        y = y.to(device)
     
        # Forward pass
        y_pred = model(x)     # (1 * 40)

        class_pred = torch.argmax(y_pred)
        # print("Pred: ", class_pred.item(), "Real: " , y.item())
        correct += int((class_pred == y).sum())

    print(f"Correct percentage : {correct / len(dataset_test)}")

Correct percentage : 0.5980551053484603


In [7]:
with torch.no_grad():
    model.eval()
    correct = 0
    for data in dataset_test:
        
        pos = data.pos        # (num_points * 3)   
        normals = data.normal # (num_points * 3)
        x = torch.cat([pos, normals], dim=1)   # (num_points * 6)
        x = x.unsqueeze(0)    # (1 * num_points * 6)     the first dimension may be used for batching?
        x = x.type(torch.float32)  # other types of data may be unstable

        y = data.y              # (1)
        y = y.type(torch.long)  # required by the loss function
        
        x = x.to(device)      # to CUDA if available
        y = y.to(device)
     
        # Forward pass
        y_pred = model(x)     # (1 * 40)

        class_pred = torch.argmax(y_pred)
        print("Pred: ", class_pred.item(), "Real: " , y.item())
        correct += int((class_pred == y_pred).sum())

    print(f"Correct percentage : {correct / len(dataset_test)}")
    

Pred:  34 Real:  29
Pred:  21 Real:  21
Pred:  19 Real:  37
Pred:  35 Real:  35
Pred:  37 Real:  11
Pred:  0 Real:  0
Pred:  2 Real:  2
Pred:  8 Real:  8
Pred:  25 Real:  33
Pred:  14 Real:  39
Pred:  0 Real:  0
Pred:  33 Real:  33
Pred:  31 Real:  21
Pred:  30 Real:  30
Pred:  0 Real:  0
Pred:  37 Real:  5
Pred:  30 Real:  30
Pred:  5 Real:  5
Pred:  28 Real:  28
Pred:  14 Real:  23
Pred:  17 Real:  17
Pred:  31 Real:  21
Pred:  36 Real:  36
Pred:  8 Real:  8
Pred:  37 Real:  37
Pred:  0 Real:  0
Pred:  20 Real:  20
Pred:  17 Real:  17
Pred:  4 Real:  4
Pred:  17 Real:  17
Pred:  33 Real:  33
Pred:  37 Real:  10
Pred:  37 Real:  10
Pred:  22 Real:  22
Pred:  33 Real:  33
Pred:  35 Real:  35
Pred:  30 Real:  30
Pred:  21 Real:  21
Pred:  37 Real:  37
Pred:  31 Real:  25
Pred:  17 Real:  17
Pred:  26 Real:  26
Pred:  31 Real:  7
Pred:  31 Real:  6
Pred:  14 Real:  14
Pred:  16 Real:  25
Pred:  4 Real:  14
Pred:  19 Real:  26
Pred:  0 Real:  0
Pred:  35 Real:  35
Pred:  0 Real:  0
Pred: 

Pred:  7 Real:  7
Pred:  17 Real:  36
Pred:  33 Real:  33
Pred:  14 Real:  14
Pred:  22 Real:  22
Pred:  17 Real:  17
Pred:  37 Real:  5
Pred:  12 Real:  12
Pred:  13 Real:  22
Pred:  7 Real:  7
Pred:  34 Real:  28
Pred:  7 Real:  7
Pred:  7 Real:  7
Pred:  26 Real:  26
Pred:  14 Real:  14
Pred:  2 Real:  2
Pred:  31 Real:  16
Pred:  33 Real:  12
Pred:  35 Real:  35
Pred:  2 Real:  1
Pred:  27 Real:  16
Pred:  31 Real:  35
Pred:  27 Real:  16
Pred:  33 Real:  33
Pred:  8 Real:  8
Pred:  30 Real:  30
Pred:  30 Real:  25
Pred:  26 Real:  26
Pred:  8 Real:  32
Pred:  26 Real:  26
Pred:  4 Real:  4
Pred:  9 Real:  19
Pred:  7 Real:  7
Pred:  9 Real:  9
Pred:  31 Real:  22
Pred:  0 Real:  0
Pred:  17 Real:  17
Pred:  31 Real:  35
Pred:  33 Real:  33
Pred:  14 Real:  14
Pred:  37 Real:  26
Pred:  2 Real:  2
Pred:  31 Real:  25
Pred:  17 Real:  17
Pred:  29 Real:  1
Pred:  33 Real:  33
Pred:  13 Real:  39
Pred:  36 Real:  36
Pred:  21 Real:  21
Pred:  21 Real:  21
Pred:  31 Real:  1
Pred:  31

Pred:  28 Real:  28
Pred:  2 Real:  2
Pred:  28 Real:  28
Pred:  8 Real:  8
Pred:  21 Real:  28
Pred:  37 Real:  37
Pred:  21 Real:  21
Pred:  0 Real:  0
Pred:  31 Real:  37
Pred:  23 Real:  33
Pred:  36 Real:  36
Pred:  36 Real:  36
Pred:  7 Real:  7
Pred:  2 Real:  2
Pred:  30 Real:  30
Pred:  31 Real:  3
Pred:  2 Real:  2
Pred:  37 Real:  11
Pred:  22 Real:  11
Pred:  17 Real:  17
Pred:  35 Real:  35
Pred:  4 Real:  16
Pred:  14 Real:  14
Pred:  5 Real:  5
Pred:  36 Real:  4
Pred:  19 Real:  1
Pred:  12 Real:  12
Pred:  25 Real:  33
Pred:  31 Real:  31
Pred:  2 Real:  2
Pred:  4 Real:  4
Pred:  12 Real:  33
Pred:  30 Real:  30
Pred:  7 Real:  7
Pred:  4 Real:  4
Pred:  25 Real:  33
Pred:  14 Real:  16
Pred:  32 Real:  23
Pred:  20 Real:  20
Pred:  31 Real:  23
Pred:  37 Real:  10
Pred:  4 Real:  4
Pred:  27 Real:  36
Pred:  14 Real:  14
Pred:  36 Real:  27
Pred:  31 Real:  21
Pred:  17 Real:  17
Pred:  31 Real:  26
Pred:  17 Real:  17
Pred:  0 Real:  0
Pred:  23 Real:  12
Pred:  30 

Pred:  23 Real:  23
Pred:  2 Real:  2
Pred:  37 Real:  24
Pred:  20 Real:  20
Pred:  21 Real:  21
Pred:  0 Real:  0
Pred:  6 Real:  37
Pred:  4 Real:  36
Pred:  26 Real:  26
Pred:  8 Real:  8
Pred:  37 Real:  26
Pred:  6 Real:  6
Pred:  17 Real:  17
Pred:  35 Real:  35
Pred:  35 Real:  35
Pred:  31 Real:  35
Pred:  37 Real:  5
Pred:  4 Real:  14
Pred:  22 Real:  28
Pred:  17 Real:  17
Pred:  29 Real:  34
Pred:  4 Real:  16
Pred:  37 Real:  37
Pred:  14 Real:  14
Pred:  39 Real:  5
Pred:  29 Real:  12
Pred:  31 Real:  26
Pred:  4 Real:  21
Pred:  26 Real:  24
Pred:  21 Real:  21
Pred:  31 Real:  35
Pred:  37 Real:  5
Pred:  2 Real:  2
Pred:  31 Real:  1
Pred:  27 Real:  16
Pred:  30 Real:  30
Pred:  4 Real:  4
Pred:  9 Real:  9
Pred:  18 Real:  18
Pred:  3 Real:  28
Pred:  37 Real:  24
Pred:  2 Real:  34
Pred:  0 Real:  0
Pred:  36 Real:  38
Pred:  6 Real:  6
Pred:  26 Real:  24
Pred:  35 Real:  35
Pred:  17 Real:  17
Pred:  3 Real:  12
Pred:  21 Real:  21
Pred:  27 Real:  34
Pred:  5 R

Pred:  8 Real:  8
Pred:  31 Real:  26
Pred:  0 Real:  0
Pred:  2 Real:  31
Pred:  6 Real:  10
Pred:  4 Real:  4
Pred:  29 Real:  25
Pred:  14 Real:  14
Pred:  33 Real:  33
Pred:  2 Real:  2
Pred:  27 Real:  16
Pred:  2 Real:  2
Pred:  16 Real:  16
Pred:  6 Real:  1
Pred:  13 Real:  13
Pred:  23 Real:  23
Pred:  14 Real:  23
Pred:  27 Real:  1
Pred:  31 Real:  28
Pred:  37 Real:  37
Pred:  27 Real:  36
Pred:  17 Real:  17
Pred:  34 Real:  34
Pred:  2 Real:  2
Pred:  26 Real:  26
Pred:  14 Real:  14
Pred:  17 Real:  12
Pred:  5 Real:  5
Pred:  35 Real:  35
Pred:  8 Real:  8
Pred:  16 Real:  12
Pred:  4 Real:  1
Pred:  0 Real:  0
Pred:  25 Real:  25
Pred:  37 Real:  37
Pred:  18 Real:  17
Pred:  12 Real:  12
Pred:  3 Real:  28
Pred:  4 Real:  4
Pred:  17 Real:  17
Pred:  30 Real:  30
Pred:  31 Real:  21
Pred:  37 Real:  26
Pred:  25 Real:  25
Pred:  36 Real:  36
Pred:  32 Real:  8
Pred:  27 Real:  2
Pred:  4 Real:  15
Pred:  0 Real:  0
Pred:  26 Real:  26
Pred:  27 Real:  16
Pred:  37 Rea

Pred:  22 Real:  22
Pred:  21 Real:  21
Pred:  0 Real:  0
Pred:  35 Real:  35
Pred:  21 Real:  21
Pred:  26 Real:  26
Pred:  31 Real:  26
Pred:  4 Real:  4
Pred:  31 Real:  2
Pred:  14 Real:  38
Pred:  21 Real:  21
Pred:  39 Real:  5
Pred:  4 Real:  36
Pred:  34 Real:  34
Pred:  30 Real:  30
Pred:  22 Real:  22
Pred:  14 Real:  23
Pred:  2 Real:  2
Pred:  31 Real:  35
Pred:  7 Real:  7
Pred:  31 Real:  31
Pred:  29 Real:  1
Pred:  4 Real:  26
Pred:  14 Real:  38
Pred:  31 Real:  25
Pred:  31 Real:  31
Pred:  19 Real:  19
Pred:  21 Real:  21
Pred:  31 Real:  27
Pred:  12 Real:  12
Pred:  27 Real:  27
Pred:  21 Real:  21
Pred:  8 Real:  8
Pred:  5 Real:  5
Pred:  25 Real:  25
Pred:  12 Real:  12
Pred:  7 Real:  7
Pred:  28 Real:  28
Pred:  34 Real:  9
Pred:  4 Real:  23
Pred:  36 Real:  36
Pred:  14 Real:  16
Pred:  21 Real:  21
Pred:  35 Real:  35
Pred:  0 Real:  0
Pred:  6 Real:  1
Pred:  17 Real:  17
Pred:  26 Real:  26
Pred:  35 Real:  35
Pred:  5 Real:  5
Pred:  14 Real:  14
Pred:  

In [8]:
with torch.no_grad():
    x = torch.rand([1, 1024, 6])*5+3
    x = x.to(device)
    y = model(x)
    y_class = torch.argmax(y)
    print(y)
    print(y_class)

tensor([[-2560.8369, -3661.9685, -1816.2803, -2752.5654, -8216.8311, -6302.7329,
         -3227.3005, -3909.5195, -2146.2859, -5782.8008, -6951.6030, -6440.3677,
         -3573.5662, -6648.8105, -5812.2671, -2965.5229, -4939.5874, -3191.7422,
         -3529.2002, -5764.1094, -2446.8389, -8682.3779, -5988.0640, -4008.8279,
         -5037.1968, -3104.1531, -2498.1562, -2789.3403, -5321.3579, -3374.8577,
         -2772.9187, -2706.5046, -2806.4038, -3464.4272, -3806.1746, -2847.5786,
         -4085.6536, -4842.5186, -9695.8975, -7117.3984]], device='cuda:0')
tensor(2, device='cuda:0')


WORK IN PROGRESS!!! 
Trying to create batches from data...

In [9]:
import numpy as np
from torch_geometric.data import Data

length = len(dataset_train)
batch_size = 32
iterations = np.ceil(length/batch_size)
iterations = iterations.astype(int)
batched_data = torch.empty([125, 1024, 6])
print(dataset_train)
print(dataset_train[0])
aux = Data()
for i in range(iterations):
    ob = dataset_train[i:i+batch_size]
    pos=torch.empty([0, 3])
    y = torch.empty([0])
    normal = torch.empty([0, 3])
    for data in ob:
        pos = torch.cat([pos, data.pos])
        y = torch.cat([y, data.y])
        normal = torch.cat([normal, data.normal])
    batch_data[i] = Data(pos=pos, y=y, normal=normal)
print(pos.shape)

ModelNet40(9843)
Data(pos=[1024, 3], y=[1], normal=[1024, 3])


NameError: name 'batch_data' is not defined

In [ ]:
#print(pos.shape)
#print(pos)
print(len(batch_data))
Batched_data = torch.empty([125, 1024, 6])
BATCHED_DATA = []
for i in range(125):
    # print(batch_data[i].pos)
    pos = torch.empty([32, 1024, 3])
    y = torch.empty([32, 1])
    normal = torch.empty([32, 1024, 3])
    for i in range(batch_size):
        pos[i] = batch_data[i].pos[num_points*i:num_points*i+1024]
        y[i] = batch_data[i].y[i]
        normal[i] = batch_data[i].normal[num_points*i:num_points*i+1024]
    BATCH = Data(pos=pos, y=y, normal=normal)
    BATCHED_DATA.append(BATCH)
    # Batched_data[i] = Data(pos=pos, y=y, normal=normal)
print(pos.shape)
print(normal.shape)
print(y.shape)
print(len(BATCHED_DATA))
for data in BATCHED_DATA:
    print(data)